import time
plc=1
time.sleep(1500*plc)         #run after 1 houre = 3600 seconds

In [1]:
# ClsStrt0/1 , Ntwrk/CNN1/CNN2 , NumClses , loadData , Rate , Btch , ClsPrcnt , AugPrcnt

In [2]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                               #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from SE_ResNeXt_1DCNN import SEResNeXt as ResNet

cwd = os.getcwd() #
fullscrn()

ModuleNotFoundError: No module named 'SE_ResNeXt_1DCNN'

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [ ]:
rate0=80
rateNet=480
cls_num=2
for i in range (0,cls_num):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=DistalPhalanxOutlineCorrect0(i)

os.chdir(cwd)
clses_lens=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecg=np.array(ecg0)
print(0,len(vars()['ecg'+str(0)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(0)]))
for i in range (1,cls_num):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print(i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)
btch=int(min(clses_lens))

In [ ]:
np.shape(ecg)

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [ ]:
smpl_rte=rate0                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])

for cls in range (0,cls_num):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])  
    
print("each class and its windows =")

cls_wndws=np.array([])
for cls in range (0,cls_num):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')

<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [ ]:
mx_wndws=int(np.max(cls_wndws)+1)   # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)
rpt_cnn_clsfctn=3                 #تعداد اجرای شبکه عصبی برای میانگین گیری هر حالت 
aug_amnt_Ttl=np.empty((0,5), int) 
rslts=np.array([])
for X in range(1,5):                               #داده افزایی تا چند برابر کلاس اکثریت
    mx_wndws=int(np.max(cls_wndws))   # حداکثر تعداد پنجره ی موجود بین کلاس ها
    mx_wndws=int(X*mx_wndws)          #
    for alpha in range(1,4):
        rprt=np.array([])
        for beta in range(1,4):                        #چند برابر کردن داده افزایی هر کلاس متناسب ضعیف بودن آن کلاس
            if beta==3:
                beta=beta*2

            f_scr=np.array([77,78,79,78,78,78])                      #UnAg,Scl.2,Mag.05,Tm.2,Gs1,frqnc2,GAN,LSTMexpgdo
            f_scr_difrnc=f_scr-f_scr[0]
            for z in range (len(f_scr_difrnc)):
                if f_scr_difrnc[z]<0:
                    f_scr_difrnc[z]=0
            f_scr_difrnc=f_scr_difrnc**beta
            f_scr_rtio=f_scr_difrnc/(sum(f_scr_difrnc))
            
            scors0=np.array([80,76])    #76,72 - 82,74 - 75,69     #Class Recalls without Augmentation
            MxMn20fscr=((max(f_scr)-f_scr[0])/20)**0.7
            Cls_amnt=1+alpha*MxMn20fscr*((100-scors0)/(np.max(100-scors0)))**beta        #Class Specific
            aug_amnt=np.empty((0,len(f_scr_rtio)-1), int) 

            for i in range (0,cls_num):
                aug_amnt=np.append(aug_amnt,[f_scr_rtio[1:]*(mx_wndws*Cls_amnt[i]-cls_wndws[i])],axis=0)
                #vlum_win=mx_wndws-cls_wndws[i]                        #میزان داده افزایی در روش های عمیق (پنجره)
            aug_amnt=np.int16(MxMn20fscr*np.round(aug_amnt))
            aug_amnt_Ttl=np.append(aug_amnt_Ttl,aug_amnt,axis=0)
            #print(np.int16(100*f_scr_rtio))
            rprt=np.empty((0,3),float)
            print('aug_amnt=\n',aug_amnt)

            for repeat in range(1,rpt_cnn_clsfctn+1):
                xtrain=np.empty((0,smpl_rte+1), float) 
                for i in range (0,cls_num):                            #تعریف آرایه ی پنجره های کلاس ها
                    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
                    if aug_amnt[i,0]>0:
                        methd=10
                        sort=-5  #-5=Dis_sam_Cls=qlty1 #-4=var_Dis=qlty2 #-3=DisSm/DisOthr=qlty3  #-2=invrs_GDO_papr=qlty4   #-1=GDO_papr=qlty5
                        dp_slct_dstrb=1            #1-sequencial 2-linear 3-beta 4-exponential 5-unfrm
                        xtrain=np.append(xtrain,srtd_data(methd,i,aug_amnt[i,0],dp_slct_dstrb,sort),axis=0)
                    if aug_amnt[i,1]>0:
                        methd=20
                        sort=-5
                        dp_slct_dstrb=1
                        xtrain=np.append(xtrain,srtd_data(methd,i,aug_amnt[i,1],dp_slct_dstrb,sort),axis=0)
                    if aug_amnt[i,2]>0:
                        methd=30
                        sort=-5
                        dp_slct_dstrb=1
                        xtrain=np.append(xtrain,srtd_data(methd,i,aug_amnt[i,2],dp_slct_dstrb,sort),axis=0)
                    if aug_amnt[i,3]>0:
                        methd=40
                        sort=-5
                        dp_slct_dstrb=1
                        xtrain=np.append(xtrain,srtd_data(methd,i,aug_amnt[i,3],dp_slct_dstrb,sort),axis=0)
                    if aug_amnt[i,4]>0:
                        methd=50
                        sort=-5
                        dp_slct_dstrb=1
                        xtrain=np.append(xtrain,srtd_data(methd,i,aug_amnt[i,4],dp_slct_dstrb,sort),axis=0)
                    ''''if aug_amnt[i,5]>0:
                        methd=60
                        sort=-5
                        dp_slct_dstrb=1
                        xtrain=np.append(xtrain,srtd_data(methd,i,aug_amnt[i,5],dp_slct_dstrb,sort),axis=0)
                    if aug_amnt[i,6]>0:
                        methd=70
                        sort=-1
                        dp_slct_dstrb=2
                        xtrain=np.append(xtrain,srtd_data(methd,i,aug_amnt[i,6],dp_slct_dstrb,sort),axis=0)'''
                    #print('class ', i, 'train shape = ', np.shape(xtrain))

                # Normalization train windows
                mx_aug=np.max(xtrain[:,:-1])
                mx=np.max(ecg[:,:-1])
                mn_aug=np.min(xtrain[:,:-1])
                mn=np.min(ecg[:,:-1])
                xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1


                ###Test Windows:
                wndws_test=np.empty((0,len(wndws1[0])), float)
                i=0
                windws=np.array([])
                for cls in range (0,cls_num):                                 #ساخت پنجره های داده های آموزش اصلی
                    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
                    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

                # Normalization test windows
                for i in range (0,cls_num):                
                    cls=i
                    #print('\n cls', i, ' >> ')
                    #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                    #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
                    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
                    #print('after normalizing >>')
                    #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                    #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
                #print("each class and its windows =")                    #تجمیع کلاس های تست
                for cls in range (0,cls_num):                                # آرایه ی تعداد پنجره ی هر کلاس
                    wns=len(vars()['wndws_tst'+str(cls)])
                    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
                Xtest=wndws_test[:,:-1]
                ytest=np.int16(wndws_test[:,-1])

                trainx=np.random.permutation(xtrain)
                Xtrain=np.array(trainx[:,:-1])
                ytrain=np.int16(trainx[:,-1])

                X_train=np.array(signal.resample(Xtrain,rateNet,axis=1))
                y_train=np.array(ytrain)
                X_test=np.array(signal.resample(Xtest,rateNet,axis=1))
                y_test=np.array(ytest)
                X_valid=np.array(signal.resample(Xtrain,rateNet,axis=1))
                y_valid=np.array(ytrain)

                if min(y_train)==1:
                    y_train=y_train-1
                    y_test=y_test-1
                    y_valid =y_valid-1
                y_train_cat=to_categorical(y_train)
                y_valid_cat=to_categorical(y_valid)
                y_test_cat=to_categorical(y_test)
                X_train = np.expand_dims(X_train, axis=2)
                X_valid = np.expand_dims(X_valid, axis=2)
                X_test = np.expand_dims(X_test, axis=2)

                ####### NETWORK #######
                #VGG16_1D

                "Configurations for ResNet in Regression Mode"
                length = X_train.shape[1]   # Number of Features (or length of the signal)
                model_width = 128           # Number of Filter or Kernel in the Input Layer
                num_channel = 1             # Number of Input Channels
                problem_type = 'Classification' # Regression or Classification
                output_number = cls_num     # Number of Outputs in the Regression Mode
                Regression_Model = ResNet(length, num_channel, model_width, problem_type=problem_type, output_nums=output_number).SEResNeXt50() # Build Model
                # SEResNext Models supported: SEResNeXt18, SEResNeXt34, SEResNeXt50, SEResNeXt101, SEResNeXt152, 
                Regression_Model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
                earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 100, restore_best_weights = True)
                history = Regression_Model.fit(X_train, y_train_cat, epochs=500, batch_size=btch, verbose=1, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping]) #,validation_split=0.2,callbacks =[callbacks]
                _, accuracy = Regression_Model.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=verbose)
                predict_x=Regression_Model.predict(X_test)              # Function 1

                y_pred_crs=np.argmax(predict_x,axis=1)           # function from augment.py to remove 0 index predictions
                rprt0=classification_report(y_test, y_pred_crs,output_dict=True)
                rprt_row=np.array([])
                rprt_row=np.append(rprt_row,rprt0['accuracy'])
                rprt_row=np.append(rprt_row,rprt0['macro avg']['f1-score'])
                rprt_row=np.append(rprt_row,rprt0['weighted avg']['f1-score'])
                rprt=np.append(rprt,[rprt_row],axis=0)
                
            #print('rprt=',rprt)
            maxm=rprt[np.argmax(rprt[:,1])]
            #print('maxm=',maxm)
            avrg=np.mean(rprt,axis=0)
            #print('avrg=',avrg)
            rslts=np.append(rslts,['X '+str(X)+'  alpha '+str(alpha) +'  Beta '+str(beta)+'  average= '+str(np.int16(100*avrg))+'  max= '+str(np.int16(100*maxm))+'  var= '+str(np.int16(np.var(100*rprt,axis=0)))],axis=0)
            print('rslts=',rslts[-1])

In [ ]:
rslts